# Predicting Hockey Contracts - The Complete Guide

by Luke Kerwin, Eric Wu, Brian Ellis, and Griffin Jordan

In [2]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Step 1: Getting the Data

We decided as a group that we were going to take on the task of predicting NHL (hockey) contracts for players in the season 2022-2023. We chose this specific season as it is the most recent completed season. In order to reach our goal, we needed to gather data on the players and their contracts. We also needed to gather data on the players' performance, such as goals, assists, points, etc. We gathered this data from the following sources:

- [CapFriendly](https://www.capfriendly.com/) - a website that tracks NHL contracts
- [Hockey Reference](https://www.hockey-reference.com/) - a website that tracks NHL player performance statistics

Below is the code we used to scrape the data from these websites. We used the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) library to scrape the data from the websites. We also used the [requests](https://docs.python-requests.org/en/master/) library to make the HTTP requests to the websites.

### CapFriendly

In [3]:
# We need to get the last 10 seasons of contracts for each player
years = range(2012, 2023)

# Iterate through each season, month by month
contracts = []
for year in years:
    for month in range(1, 13):
        print(f'Getting contract data for {year}-{month}...', end='\r')
        # Get the data from capfriendly
        url = f'https://www.capfriendly.com/signings/all/all/all/1-15/0-15000000/{month}01{year}-{month+1}31{year}'
        r = requests.get(url)
        table = pd.read_html(r.text)[0]
        # Add the data to our list
        contracts.append(table)

contracts = pd.concat(contracts)
contracts.to_csv('data/contracts.csv', index=False)
print('\nDone!')
contracts.head()
        

Getting contract data for 2022-12...
Done!


,PLAYER,PLAYER.1,AGE,POS,TEAM,DATE,TYPE,EXTENSION,STRUCTURE,LENGTH,VALUE,CAP HIT
0,Jordan Gustafson,Jordan Gustafson,19,C,VGK,"Nov. 29, 2023",ELC,NaN,2-way,3,"$2,850,000","$950,000"
1,Patrick Kane,Patrick Kane,34,RW,DET,"Nov. 28, 2023",Stnd (UFA),NaN,1-way,1,"$2,750,000","$2,750,000"
2,Justin Bailey,Justin Bailey,27,RW,SJS,"Nov. 27, 2023",Stnd (UFA),NaN,2-way,1,"$775,000","$775,000"
3,Ben Hemmerling,Ben Hemmerling,19,RW,VGK,"Nov. 26, 2023",ELC,NaN,2-way,3,"$2,532,500","$844,167"
4,Samuel Laberge,Samuel Laberge,26,C,NJD,"Nov. 25, 2023",Stnd (UFA),NaN,2-way,1,"$775,000","$775,000"


### Hockey Reference

We are going to use the 3 seasons of statistics before the player is awarded their contract to predict the contract.

In [4]:
stats_seasons = range(2009, 2023)

stats = []
for season in stats_seasons:
    print(f'Getting stats for {season}...', end='\r')
    req = requests.get(f'https://www.hockey-reference.com/leagues/NHL_{season}_skaters.html')
    table = pd.read_html(req.text)[0]
    table['season'] = season
    stats.append(table)

In [5]:
stats_ = pd.concat(stats)
# Remove the multi-index
stats_.columns = ['Rk', 'Player', 'Age', 'Tm', 'Pos', 'GP', 'G', 'A', 'PTS', '+/-', 'PIM',
       'PS', 'EV', 'PP', 'SH', 'GW', 'EV', 'PP', 'SH', 'S', 'S%', 'TOI',
       'ATOI', 'BLK', 'HIT', 'FOW', 'FOL', 'FO%', 'SEASON']
stats_.to_csv('data/stats.csv', index=False)
stats_.head()

,Rk,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,...,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%,SEASON
0,1,Justin Abdelkader,21,DET,LW,2,0,0,0,0,...,2,0.0,19,9:18,0,3,4,3,57.1,2009
1,2,Craig Adams,31,TOT,RW,45,2,5,7,-3,...,47,4.3,391,8:41,20,67,8,13,38.1,2009
2,2,Craig Adams,31,CHI,RW,36,2,4,6,-3,...,38,5.3,314,8:43,16,53,6,10,37.5,2009
3,2,Craig Adams,31,PIT,RW,9,0,1,1,0,...,9,0.0,77,8:34,4,14,2,3,40.0,2009
4,3,Maxim Afinogenov,29,BUF,RW,48,6,14,20,-7,...,93,6.5,605,12:36,11,20,0,3,0.0,2009


# Step 2: Cleaning and Preparing the Data for Analysis

Now that we have the data, we need to clean it and prepare it for analysis. As I mentioned above, we will want to use the 3 previous seasons of data to predict the contract, so we will have to merge on the contract data but only use the previous 3 seasons of performance data.


### Cleaning Contract Data

In [6]:
# Remove PLAYER.1
cleaned_contracts = contracts.drop(columns=['PLAYER.1'])
# Data formatting
cleaned_contracts['DATE'] = pd.to_datetime(cleaned_contracts['DATE']).dt.year
cleaned_contracts['VALUE'] = cleaned_contracts['VALUE'].str.replace('$', '').str.replace(',', '').astype(float)
cleaned_contracts['CAP HIT'] = cleaned_contracts['CAP HIT'].str.replace('$', '').str.replace(',', '').astype(float)
cleaned_contracts['EXTENSION'] = cleaned_contracts['EXTENSION'].str.replace('✔', '1').fillna(0).astype(int)

# Removing data we dont want
cleaned_contracts = cleaned_contracts[cleaned_contracts['TYPE'].isin(['Stnd (UFA)','35+ (UFA)'])]
cleaned_contracts = cleaned_contracts[cleaned_contracts['EXTENSION']==0].reset_index(drop=True)
cleaned_contracts.to_csv('data/cleaned_contracts.csv', index=False)
cleaned_contracts.head()

,PLAYER,AGE,POS,TEAM,DATE,TYPE,EXTENSION,STRUCTURE,LENGTH,VALUE,CAP HIT
0,Patrick Kane,34,RW,DET,2023,Stnd (UFA),0,1-way,1,2750000.0,2750000.0
1,Justin Bailey,27,RW,SJS,2023,Stnd (UFA),0,2-way,1,775000.0,775000.0
2,Samuel Laberge,26,C,NJD,2023,Stnd (UFA),0,2-way,1,775000.0,775000.0
3,Joel Kiviranta,27,"RW, LW",COL,2023,Stnd (UFA),0,2-way,1,775000.0,775000.0
4,Sam Gagner,33,"RW, C",EDM,2023,Stnd (UFA),0,2-way,1,775000.0,775000.0


### Cleaning Statistics Data

In [26]:
stats_cleaned = stats_.copy()
stats_cleaned = stats_cleaned[['Player', 'Age', 'Tm', 'Pos', 'GP', 'G', 'A', 'PTS', '+/-', 'PIM',
       'PS', 'EV', 'PP', 'SH', 'GW', 'EV', 'PP', 'SH', 'S', 'S%', 'TOI',
       'ATOI', 'BLK', 'HIT', 'FOW', 'FOL', 'FO%', 'SEASON']]
stats_cleaned = stats_cleaned[stats_cleaned['Player']!='Player'].reset_index(drop=True)

new = []
for player in stats_cleaned['Player'].unique():
    data = stats_cleaned[stats_cleaned['Player']==player]
    for season in data['SEASON'].unique():
        data_season = data[data['SEASON']==season]
        if len(data_season) > 1:
            data_season = data_season[data_season['Tm']=='TOT']
        else:
            pass
        new.append(data_season)

stats_cleaned = pd.concat(new).reset_index(drop=True)
stats_cleaned['Pos'] = np.where(stats_cleaned['Pos'].isin(['D','LD','RD']), 'D', np.where(stats_cleaned['Pos'].isin(['C']), 'C', 'W'))
stats_cleaned['Pos'].unique()
stats_cleaned.columns = ['PLAYER', 'AGE', 'TEAM', 'POS', 'GP', 'G', 'A', 'PTS', 'PLUSMINUS', 'PIM', 'PS',
       'EVG', 'EVA', 'PPG', 'PPA', 'EVSH', 'PPSH', 'GWG', 'EV', 'EV', 'PP', 'PP', 'SH',
       'SH', 'S', 'S%', 'TOI', 'ATOI', 'BLK', 'HIT', 'FOW', 'FOL', 'FO%',
       'SEASON']
stats_cleaned = stats_cleaned.drop(columns=['EV', 'PP', 'SH'])
stats_cleaned['S%'] = stats_cleaned['S%'].str.replace('%', '').astype(float)/100
stats_cleaned['TOI'] = stats_cleaned['TOI'].astype(int) * 60
stats_cleaned['ATOI'] = stats_cleaned['ATOI'].str.split(':').apply(lambda x: int(x[0])*60 + int(x[1]))
stats_cleaned['FO%'] = stats_cleaned['FO%'].str.replace('%', '').astype(float)/100
stats_cleaned = stats_cleaned[stats_cleaned['GP'].astype(int)>=20].reset_index(drop=True)

for col in stats_cleaned.columns:
    try:
        stats_cleaned[col] = stats_cleaned[col].astype(float)
    except:
        try:
            stats_cleaned[col] = stats_cleaned[col].astype(int)
        except:
            try:
                stats_cleaned[col] = stats_cleaned[col].astype(str)
            except:
                pass

stats_cleaned.to_csv('data/stats_cleaned.csv', index=False)

# Step 3: Creating SQL Database

In [27]:
from sqlalchemy import create_engine

# Create the database
engine = create_engine('sqlite:///data/nhl.db', echo=False)

# Save the dataframes to the database
cleaned_contracts.to_sql('contracts', con=engine)
stats_cleaned.to_sql('stats', con=engine)

9393

# Step 4: Building the API

In [ ]:
from flask import Flask

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///data/nhl.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

from flask_sqlalchemy import SQLAlchemy
db = SQLAlchemy(app)

class Contract(db.Model):
    __tablename__ = 'contracts'
    index = db.Column(db.Integer, primary_key=True)
    PLAYER = db.Column(db.String)
    TEAM = db.Column(db.String)
    TYPE = db.Column(db.String)
    DATE = db.Column(db.Integer)
    VALUE = db.Column(db.Float)
    LENGTH = db.Column(db.Integer)
    CAP_HIT = db.Column(db.Float)
    EXTENSION = db.Column(db.Integer)

    def __repr__(self):
        return '<Contract %r>' % self.PLAYER
    
class Stat(db.Model):
    __tablename__ = 'stats'
    index = db.Column(db.Integer, primary_key=True)
    PLAYER = db.Column(db.String)
    AGE = db.Column(db.Float)
    TEAM = db.Column(db.String)
    POS = db.Column(db.String)
    GP = db.Column(db.Float)
    G = db.Column(db.Float)
    A = db.Column(db.Float)
    PTS = db.Column(db.Float)
    PLUSMINUS = db.Column(db.Float)
    PIM = db.Column(db.Float)
    PS = db.Column(db.Float)
    EVG = db.Column(db.Float)
    EVA = db.Column(db.Float)
    PPG = db.Column(db.Float)
    PPA = db.Column(db.Float)
    EVSH = db.Column(db.Float)
    PPSH = db.Column(db.Float)
    GWG = db.Column(db.Float)
    S = db.Column(db.Float)
    S_ = db.Column(db.Float)
    TOI = db.Column(db.Float)
    ATOI = db.Column(db.Float)
    BLK = db.Column(db.Float)
    HIT = db.Column(db.Float)
    FOW = db.Column(db.Float)
    FOL = db.Column(db.Float)
    FO_ = db.Column(db.Float)
    SEASON = db.Column(db.Float)

    def __repr__(self):
        return '<Stat %r>' % self.PLAYER

from flask import jsonify

@app.route('/api/contracts')
def get_contracts():
    contracts = Contract.query.all()
    return jsonify([contract.__dict__ for contract in contracts])

@app.route('/api/stats')
def get_stats():
    stats = Stat.query.all()
    return jsonify([stat.__dict__ for stat in stats])